### R-CNN（Regional Convolutional Neural Networks，区域卷积神经网络）
> * 使用<font color='red'>启发式搜索（Selective search）算法</font>来选择锚框
> * 是用预训练模式来对每个锚框抽取特征(<font color='red'>通过预训练的卷积神经网络抽取每个锚框的特征</font>)
> * 训练一个SVM来对类别分类
> * 训练一个线性回归模型来预测边缘框偏移

#### R-CNN步骤如下：
> * 对输入图像使用选择性搜索来选取多个高质量的提议趋于。这些提议区域通常是在多个尺度下选取的，并具有不同的形状和大小。每个提议区域都将被标注类别和真实边界框
> * 选择一个预训练的卷积神经网络，并将其在输出层之前截断。将每个提议区域变形为网络需要的输入尺寸，并通过前向传播输出抽取的提议区域特征
> * 将每个提议区域特征连同其标注的类别作为一个样本。训练<font color='red'>多个支持向量机对目标分类</font>，其中每个支持向量机用来判断样本是否属于同一个类别
> * 将每个提议区域的特征连同标注的边界框作为一个样本，训练<font color='red'>线性回归模型来预测真实边界框</font>
![](../images/R-CNN.png)

<font color='red'>尽管R-CNN模型通过预训练的卷积神经网络有效的抽取了图像特征,但它的速度很慢。假如从一张图片中选出了上千个提议区域，这需要上千次卷积神经网络的前向传播来执行目标检测，计算量过于庞大。</font>

### 兴趣区域（RoI）池化层
> * 给定一个锚框，均匀分成nxm块，输出每块里的最大值
> * 不管锚框多大，总是输出nm个值

#### RoI过程如下：
> 假设指定每个区域输出的宽和高分别为$h_2$和$w_2$。对于任何形状为$h\times w$的兴趣区域窗口，该窗口被划分为$h_2\times w_2$子窗口网格，其中每个子窗口的大小约为$(h/h_2)\times w/w2$。在实践中，任何子窗口的高度和宽度都应该向上取整，其中最大元素作为该子窗口的输出。因此<font color='red'>兴趣区域汇聚层可从形状各异的兴趣区域中均抽取出形状相同的特征</font>。
#### 说明实例：
> 在$4\times4$的输入中，选取左上角$3\times3$的感兴趣区域。对于该区域，通过$2\times2$的兴趣区域汇聚层得到一个$2\times2$的输出。请注意，<font color='red'>四个划分后的子窗口中分别含有元素[0, 1, 4, 5]（5最大），[2,6]（6最大），[8,9]（9最大），[10]（10最大）</font>
![](../images/RoI.png)

### Fast R-CNN
> * 使用CNN对图像抽取特征
> * 使用RoI池化层对每个锚框生成固定长度的特征

#### Fast R-CNN计算步骤如下：
> * 与R-CNN相比，Fast R-CNN用来提取特征的卷积神经网络的输入整个图像，而不是各个提议区域，此外，这个网络通常会参与训练。设输入为一张图像，将卷积神经网络的输出形状记为$1\times c\times h_1\times w_1$；
> * 假设选择性搜索生成了n个提议区域。这些形状各异的提议区域在卷积神经网络的输出上分别标出了形状各异的兴趣区域。然后，这些感兴趣的区域需要进一步抽取形状相同的特征（比如指定的高度$h_2$和宽度$w_2$），以便于连接后输出。为了实现这一目标，Fast R-CNN引入了兴趣区域汇聚层（RoI pooling）：将卷积神经网络的输出和提议区域作为输入，输出连接后的各个提议区域抽取的特征，形状为$n\times c\times h_2\times w_2$；
> * 通过全连接层将输出形状变换为$n\times d$，其中超参数$d$取决于模型设计；
> * 预测n个提议区域中每个区域的类别和边界框。更具体地说，在预测类别和边界框时，将全连接层的输出分别转换为形状为$n\times q$（q是类别的数量）的输出和形状为$n\times 4$的输出。其中预测类别时使用softmax回归。
> ![](../images/FastR-CNN.png)

### Faster R-CNN
> * 使用一个区域提议网络（Region proposal network）来代替启发式搜索来获得更好的锚框
> ![](../images/FasterR-CNN.png)
#### 区域提议网络（RpNet）计算步骤
> * 使用填充为1的$3\times3$的卷积层变换卷积神经网络的输出，并将输出通道记为$c$。这样，卷积神经网络为图像抽取的特征图中的每个单元均得到一个长度为c的新特征
> * 以特征图的每个像素为中心，生成多个不同大小和宽高比的锚框并标注它们
> * 使用锚框中心单元长度为$c$的特征，分别预测该锚框的二元分类（含目标还是背景）和边界框。
> * 使用非极大值抑制，从预测类别为目标的预测边界框中移除相似的结果。最终输出的预测边界框即是兴趣区域汇聚层所需要的提议区域

<font color='red'>区域提议网络能够学习到如何生成高质量的提议区域，从而在减少了从数据中学习的提议区域的数量的情况下，仍保持了目标检测的精度。</font>

### Mask R-CNN
> * 使用有像素级别的标号，使用FCN来利用这些信息
> ![](../images/MaskR-CNN.png)
> * Mask R-CNN将兴趣区域汇聚层替换成了兴趣区域对齐层，使用双线性插值来保留特征图上的空间信息，从而更适应于像素级预测。
> * 兴趣区域对齐层的输出包含了所有与兴趣区域的形状相同的特征图。它们不仅被用于预测每个兴趣区域的类别和边界框，还可以通过额外的全卷积网络预测目标的像素级位置。

### 总结
> * R-CNN是最早、也是最有名的一类基于锚框和CNN的目标检测算法
> * Fast/Faster R-CNN持续提升性能
> * Faster R-CNN和Mask R-CNN是在要求<font color='red'>高精度</font>场景下的常用算法，因为这两个算法速度很慢

### 单发多框检测（SSD）
#### 生成锚框
> * 对每个像素，生成多个以它为中心的锚框
> * 给定n个大小$s_1,s_2,...,s_n$和m个高宽比，生成n+m-1个锚框，其大小和高宽比分别为：$(s_1,r_1),(s_2,r_1),...,(s_n,r_1),(s_1,r_2),...,(s_1,r_m)$
#### SSD模型
> * 一个基础网络来抽取特征，然后多个卷积层块来减半高宽
> * 在每段都生成锚框
>> * 底部段来拟合小物体，顶部段来拟合大物体
> * 对每个锚框预测类别和边缘框
> ![](../images/SSD.png)
#### 总结
> * SSD通过单神经网络来检测模型
> * 以每个像素为中心的产生多个锚框
> * 在多个段的输出上进行多尺度的检测

### YOLO
> * SSD中锚框大量重叠，因此浪费了很多计算
> * YOLO将图片均匀分成$S\times S$个锚框
> * 每个锚框预测B个边缘框
> * 后续版本（V2，V3，V4...）有持续改进